In [24]:
import hopsworks
import xgboost as xgb
import unicodedata
import re
from xgboost import XGBRegressor

# connect with Hopsworks
project = hopsworks.login(
        host="eu-west.cloud.hopsworks.ai",
        project="ID2223_Project",
        api_key_value=os.environ["HOPSWORKS_API_KEY"]
    )

# Get feature view
fs = project.get_feature_store()
fv = fs.get_feature_view('avalanche_warning_fv_new_corrected_more_features_and_lags', version=5)

# Get model registry
mr = project.get_model_registry()

2026-01-10 01:14:18,694 INFO: Closing external client and cleaning up certificates.
2026-01-10 01:14:18,697 INFO: Connection closed.
2026-01-10 01:14:18,698 INFO: Initializing external client
2026-01-10 01:14:18,698 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-10 01:14:20,134 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/2173


In [51]:
def sanitize_name(name):
    # Normalize Unicode to ASCII, ignore accents
    name_ascii = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode()
    # Replace anything not a-z, A-Z, 0-9, or _ with underscore
    name_clean = re.sub(r'[^a-zA-Z0-9_]', '_', name_ascii)
    return name_clean

def predict(model: xgb.XGBRegressor, features_df: pd.DataFrame) -> float:
    """
    Predict avalanche risk
    """
    features_df = features_df.astype(float)
    return float(model.predict(features_df)[0])

In [35]:
# Create batch data for the feature view
batch_data = fv.get_batch_data(dataframe_type="pandas")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.93s) 


In [34]:
from xgboost import XGBRegressor

#Retrieve the name resorts
resorts = {loc: None for loc in batch_data["location"].unique()}

models = {}           
model_dirs = {}      

for loc in resorts.keys():  
    loc_ = sanitize_name(loc.replace(" ", "_"))
    print(f"Loading model for {loc}...")

    # Retrieve model from registry
    model = mr.get_model(
        name=f"xgb_avalanche_model_{loc_}",
        version=3  
    )

    print(model)
    
    # Download model artifacts
    model_dir = model.download()

    # Load XGBoost model
    xgb_model = XGBRegressor()
    xgb_model.load_model(
        f"{model_dir}/xgb_ordinal_model_more_features{loc_}.json"
    )

    # Store everything
    models[loc] = xgb_model
    model_dirs[loc] = model_dir

    print(f"✓ Model for {loc} loaded successfully\n")


Loading model for Bjorli Ski...
Model(name: 'xgb_avalanche_model_Bjorli_Ski', version: 3)


Downloading: 0.000%|          | 0/399508 elapsed<00:00 remaining<?

✓ Model for Bjorli Ski loaded successfullys)... DONE

Loading model for Eikedalen Ski Center AS...
Model(name: 'xgb_avalanche_model_Eikedalen_Ski_Center_AS', version: 3)


Downloading: 0.000%|          | 0/366544 elapsed<00:00 remaining<?

✓ Model for Eikedalen Ski Center AS loaded successfully

Loading model for Galdhøpiggen Summer Ski Centre...
Model(name: 'xgb_avalanche_model_Galdhpiggen_Summer_Ski_Centre', version: 3)


...

✓ Model for Sauda Ski Centre loaded successfullyDONE

Loading model for Strandafjellet Skisenter...
Model(name: 'xgb_avalanche_model_Strandafjellet_Skisenter', version: 3)


Downloading: 0.000%|          | 0/366828 elapsed<00:00 remaining<?

✓ Model for Strandafjellet Skisenter loaded successfully

Loading model for Voss Resort Fjellheisar...
Model(name: 'xgb_avalanche_model_Voss_Resort_Fjellheisar', version: 3)


Downloading: 0.000%|          | 0/601204 elapsed<00:00 remaining<?

✓ Model for Voss Resort Fjellheisar loaded successfully



In [76]:
# Sort batch data by date
batch_data_sorted = batch_data.sort_values(
    by="date", 
    ascending=False
)

# DataFrame of resorts considering only the most recent data
latest_per_location = (
    batch_data_sorted
    .drop_duplicates(subset="location", keep="first")
)

# Dictionary to hold a DataFrame for each location
dfs_per_location = {}

# Loop over unique locations
for i, location in enumerate(latest_per_location['location'].unique()):
    loc_ = sanitize_name(location)
    # Create a copy for the dictionary
    dfs_per_location[location] = latest_per_location[latest_per_location['location'] == location].copy()
    
    # Dynamically create a variable for each dataframe
    globals()[f'df_location_{loc_}'] = dfs_per_location[location]

# Feature columns
feature_cols = [
    "warning_level_lag_1", 
    "warning_level_lag_2",
    "warning_level_lag_3",
    "temperature_2m_mean",
    "precipitation_sum",
    "rain_sum",
    "snowfall_sum",
    "wind_speed_10m_max",
    "wind_direction_10m_dominant",
    "snow_load_steep",
    "wind_snow_transport",
    "rain_on_snow_risk",
    "temp_elev",
    "precip_slope_weighted",
]

In [89]:
for location in resorts.keys():
    loc_ = sanitize_name(location.replace(" ", "_"))

    df_name = f'df_location_{loc_}' 

    features = globals()[df_name][feature_cols]
    
    prediction = predict(models[location], features)

    for i in range(7):
        # Assign lags to each day
        globals()[df_name].loc[i, 'warning_level_lag_1'] = lag_1
        globals()[df_name].loc[i, 'warning_level_lag_2'] = lag_2
        globals()[df_name].loc[i, 'warning_level_lag_3'] = lag_3

        df_name = f'df_location_{loc_}' 

        features = globals()[df_name][feature_cols]
    
        prediction = predict(models[location], features)

        # Update lags
        lag_3, lag_2, lag_1 = lag_2, lag_1, prediction

        



,location,date,warning_level_lag_1,warning_level_lag_2,warning_level_lag_3,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,wind_speed_10m_max,wind_direction_10m_dominant,snow_load_steep,wind_snow_transport,rain_on_snow_risk,temp_elev,precip_slope_weighted
17948,Bjorli Ski,2026-01-09 00:00:00+00:00,2.000000,2.000000,2.000000,-15.872001,0.0,0.0,0.0,7.386582,133.02507,0.0,1.096934,0.0,-10.50418,0.0
0,NaN,NaT,2.071979,2.071979,2.071979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,2.036374,2.071979,2.071979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,2.036374,2.036374,2.071979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,2.036374,2.036374,2.036374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,2.036374,2.036374,2.036374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaT,2.036374,2.036374,2.036374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaT,2.036374,2.036374,2.036374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
